# Provjera i čišćenje train dataset-a


Uvozimo potrebne biblioteke i otvaramo procesirani dataset.

In [4]:
import os, sys
import pandas as pd
from pathlib import Path
curr = Path.cwd().parent.parent
file = str(curr)
file += "/data_process/train_processed.parquet"

In [5]:
df = pd.read_parquet(file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31634 entries, 0 to 31633
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   reservation_id      31634 non-null  int64         
 1   stay_date           31634 non-null  datetime64[ns]
 2   guest_id            31634 non-null  int64         
 3   guest_country_id    31634 non-null  object        
 4   reservation_status  31634 non-null  object        
 5   reservation_date    31634 non-null  datetime64[ns]
 6   date_from           31634 non-null  datetime64[ns]
 7   date_to             31634 non-null  datetime64[ns]
 8   resort_id           31634 non-null  int64         
 9   cancel_date         6151 non-null   object        
 10  room_cnt            31634 non-null  int64         
 11  adult_cnt           31634 non-null  int64         
 12  children_cnt        31634 non-null  int64         
 13  price               27860 non-null  float64   

## Brisanje podataka kojima je datum dolaska prije datuma kreiranja rezervacija

In [6]:
df = df[df["date_from"] >= df["reservation_date"]]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31147 entries, 0 to 31633
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   reservation_id      31147 non-null  int64         
 1   stay_date           31147 non-null  datetime64[ns]
 2   guest_id            31147 non-null  int64         
 3   guest_country_id    31147 non-null  object        
 4   reservation_status  31147 non-null  object        
 5   reservation_date    31147 non-null  datetime64[ns]
 6   date_from           31147 non-null  datetime64[ns]
 7   date_to             31147 non-null  datetime64[ns]
 8   resort_id           31147 non-null  int64         
 9   cancel_date         6040 non-null   object        
 10  room_cnt            31147 non-null  int64         
 11  adult_cnt           31147 non-null  int64         
 12  children_cnt        31147 non-null  int64         
 13  price               27383 non-null  float64       


## Brisanje greska kada je datum kreiranja poslije datuma brisanja rezervacije

In [7]:
df = df[ (df["reservation_date"] <= df["cancel_date"] ) | (df["cancel_date"].isna())]
a = set(df["cancel_date"])
df.info()
#print(a)

<class 'pandas.core.frame.DataFrame'>
Index: 31147 entries, 0 to 31633
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   reservation_id      31147 non-null  int64         
 1   stay_date           31147 non-null  datetime64[ns]
 2   guest_id            31147 non-null  int64         
 3   guest_country_id    31147 non-null  object        
 4   reservation_status  31147 non-null  object        
 5   reservation_date    31147 non-null  datetime64[ns]
 6   date_from           31147 non-null  datetime64[ns]
 7   date_to             31147 non-null  datetime64[ns]
 8   resort_id           31147 non-null  int64         
 9   cancel_date         6040 non-null   object        
 10  room_cnt            31147 non-null  int64         
 11  adult_cnt           31147 non-null  int64         
 12  children_cnt        31147 non-null  int64         
 13  price               27383 non-null  float64       


## Brisanje podataka gdje je broj odraslih 0

In [8]:
df = df[df["adult_cnt"] > 0]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31147 entries, 0 to 31633
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   reservation_id      31147 non-null  int64         
 1   stay_date           31147 non-null  datetime64[ns]
 2   guest_id            31147 non-null  int64         
 3   guest_country_id    31147 non-null  object        
 4   reservation_status  31147 non-null  object        
 5   reservation_date    31147 non-null  datetime64[ns]
 6   date_from           31147 non-null  datetime64[ns]
 7   date_to             31147 non-null  datetime64[ns]
 8   resort_id           31147 non-null  int64         
 9   cancel_date         6040 non-null   object        
 10  room_cnt            31147 non-null  int64         
 11  adult_cnt           31147 non-null  int64         
 12  children_cnt        31147 non-null  int64         
 13  price               27383 non-null  float64       


## Brisanje onih kojima je datum otkazivanja nakon datuma odjave

In [9]:
df = df[ (df["cancel_date"] < df["date_to"]) | (df["cancel_date"].isna())  ]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31040 entries, 0 to 31633
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   reservation_id      31040 non-null  int64         
 1   stay_date           31040 non-null  datetime64[ns]
 2   guest_id            31040 non-null  int64         
 3   guest_country_id    31040 non-null  object        
 4   reservation_status  31040 non-null  object        
 5   reservation_date    31040 non-null  datetime64[ns]
 6   date_from           31040 non-null  datetime64[ns]
 7   date_to             31040 non-null  datetime64[ns]
 8   resort_id           31040 non-null  int64         
 9   cancel_date         5933 non-null   object        
 10  room_cnt            31040 non-null  int64         
 11  adult_cnt           31040 non-null  int64         
 12  children_cnt        31040 non-null  int64         
 13  price               27366 non-null  float64       


## Brisanje ponovljenih podataka

Iako nam je u dataset-u dan poseban podatak za svaki dan kad je gost odsjedao, ostaviti ćemo samo jedinstvene podatke i iz njih odrediti boravak pomoću datuma dolaska i datuma odlaska. Kako se nećemo baviti procjenama potrošnje nećemo izgubiti na interpretabilnosti. 

In [10]:
df.drop_duplicates(subset = "reservation_id", inplace = True, keep = "first")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14824 entries, 0 to 31633
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   reservation_id      14824 non-null  int64         
 1   stay_date           14824 non-null  datetime64[ns]
 2   guest_id            14824 non-null  int64         
 3   guest_country_id    14824 non-null  object        
 4   reservation_status  14824 non-null  object        
 5   reservation_date    14824 non-null  datetime64[ns]
 6   date_from           14824 non-null  datetime64[ns]
 7   date_to             14824 non-null  datetime64[ns]
 8   resort_id           14824 non-null  int64         
 9   cancel_date         2764 non-null   object        
 10  room_cnt            14824 non-null  int64         
 11  adult_cnt           14824 non-null  int64         
 12  children_cnt        14824 non-null  int64         
 13  price               12845 non-null  float64       


## Spremanje čistih podataka

In [11]:
df.to_csv("train_data_cleaned.csv")
df.to_parquet("train_data_cleaned.parquet")